In [1]:
library(SingleCellExperiment)
library(scater)
options(stringsAsFactors = FALSE)

Loading required package: SummarizedExperiment
Loading required package: GenomicRanges
Loading required package: stats4
Loading required package: BiocGenerics
Loading required package: parallel

Attaching package: ‘BiocGenerics’

The following objects are masked from ‘package:parallel’:

    clusterApply, clusterApplyLB, clusterCall, clusterEvalQ,
    clusterExport, clusterMap, parApply, parCapply, parLapply,
    parLapplyLB, parRapply, parSapply, parSapplyLB

The following objects are masked from ‘package:stats’:

    IQR, mad, sd, var, xtabs

The following objects are masked from ‘package:base’:

    anyDuplicated, append, as.data.frame, cbind, colMeans, colnames,
    colSums, do.call, duplicated, eval, evalq, Filter, Find, get, grep,
    grepl, intersect, is.unsorted, lapply, lengths, Map, mapply, match,
    mget, order, paste, pmax, pmax.int, pmin, pmin.int, Position, rank,
    rbind, Reduce, rowMeans, rownames, rowSums, sapply, setdiff, sort,
    table, tapply, union, unique, unsp

In [ ]:
molecules <- read.table("tung/molecules.txt", sep = "\t")  # UMIs?
anno <- read.table("tung/annotation.txt", sep = "\t", header=TRUE)
# read_counts <- read.table("tung/reads.txt", sep = "\t")    
#head(molecules[,1:3])
head(anno)

In [ ]:
## Standardise the analysis ##
umi <- SingleCellExperiment(
    assays = list(counts = as.matrix(molecules)),
    colData = anno
)

# First, filter out genes not found in any cell
genes_summed_gt_zero <- rowSums(counts(umi) > 0)
gene_names_gt_zero <- (genes_summed_gt_zero > 0)

umi_filt_genes <- umi[gene_names_gt_zero,]
message("num Genes removed = ", dim(umi) - dim(umi_filt_genes))
umi <- umi_filt_genes

# Second, define control features (genes). 
# - ERCC spike-ins or mitochondrial genes
# This can be defined either from specific gene names

user_spike_filterwords = c("ERCC")
user_spike_samples = c(
    "ENSG00000198899", "ENSG00000198727", "ENSG00000198888",
    "ENSG00000198886", "ENSG00000212907", "ENSG00000198786",
    "ENSG00000198695", "ENSG00000198712", "ENSG00000198804",
    "ENSG00000198763", "ENSG00000228253", "ENSG00000198938", "ENSG00000198840")


# populate a list of feature controls
feat_conts <- list()

for (u in 1:length(user_spike_filterwords)){
    usk <- user_spike_filterwords[u]
    usk_term <- sprintf("^%s-", usk)
    isSpike(umi, usk) <- grepl(usk_term, rownames(umi))
 
    # append
    vv <- c(feat_conts, isSpike(umi, usk))
}

isSpike(umi, "MT") <- rownames(umi) %in% user_spike_samples

umi <- calculateQCMetrics(umi,       # feature_controls = feat_conts)
    feature_controls = feat_conts
)

In [ ]:
# Cell QC -- we need to decide the size of the library
#  (how many molecules detected per sample).
#
# In the case of UMIs, this is the number of umi counts, as opposed to
#  using the number of read counts.

# Distribution of count numbers assosciated with a given UMI well
# -- we can use this histogram to filter out wells with low read-depths
#    and ensure sufficient 

user_thresh = 25000

hist(umi$total_counts, breaks = 100)
abline(v=user_thresh, col="red")

below_red <- sum(umi$total_counts < user_thresh)
above_red <- sum(umi$total_counts >= user_thresh)

message("below thresh = ", below_red, ", above thresh = ", above_red)
# So we should remove 46 cells with low amount of read-depth

In [ ]:
# We can also look at the distribution of unique gene coverage
#  in each sample

# Here we are using high-depth coverage data, so we expect a high level of coverage
#  for all detected genes in our set (this might NOT be that case for rare cell/droplet
#  data).

hist(umi$total_features, breaks = 100)
# From this we can see that most cells have around ~ 7000-10000 genes
#  which is normal for high-depth scRNA-seq.

# we need to make a way to auto-detect this.
autodetected_thresh = 7000
abline(v = autodetected_thresh, col="red")

# CAUTION: This is only true for high-depth scRNA-seq. For lower-depth
#          protocols such as droplet-based that are more tailed for
#          rare cell, we would expect a lower distribution of reads.

below_red <- sum(umi$total_features < autodetected_thresh)
above_red <- sum(umi$total_features >= autodetected_thresh)

message("below thresh = ", below_red, ", above thresh = ", above_red)
# So we should remove 116 cells which have low range of genes detected


In [ ]:
plotPhenoData(
    umi,
    aes_string(
        x = "total_features",
        y = "pct_counts_MT",
        colour = "batch"
    )
)